In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
# Define hyperparameters for the training and optimization:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 0.01
momentum = 0.5
log_interval = 10
# We will set a fixed random seed to ensure reproducibility of training.
torch.manual_seed(1)


def generate_training_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

def generate_testing_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])
def generate_training_transforms():
    return transforms.Compose([
        transforms.RandomRotation(degrees=10),  # Random rotation of +/- 10 degrees
        transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),  # Random crop and resize back to 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0))
    ])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=generate_training_transforms())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=generate_testing_transforms())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

example = next(iter(trainloader))

type(example)

print("Example is of length {}".format(len(example)))
print("Type of first argument is {}".format(type(example[0])))
print("Type of second argument is {}".format(type(example[1])))

print("First argument has shape: {}".format(example[0].shape))
print("Second argument has shape {}".format(example[1].shape))
def train_epoch(network, trainloader, optimizer):
    network.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for data, target in trainloader:
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_samples += target.size(0)
        probabilities = F.softmax(output, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1)
        correct_predictions += (predicted_classes == target).sum().item()

    average_loss = total_loss / len(trainloader)
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

def test_epoch(network, testloader):
      network.eval()
      total_loss = 0
      correct_predictions = 0
      total_samples = 0
      with torch.no_grad():
        for data, target in testloader:
            output = network(data)
            loss = F.cross_entropy(output, target)
            total_loss += loss.item()
            total_samples += target.size(0)
            probabilities = F.softmax(output, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1)
            correct_predictions += (predicted_classes == target).sum().item()
            average_loss = total_loss / len(testloader)
            accuracy = correct_predictions / total_samples
            return average_loss, accuracy
            test_loss, test_accuracy = test_epoch(network, testloader)
            print(f'Epoch {epoch}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}')
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.relu2 = nn.ReLU()

        feature_size = 8 * 8

        self.fc = nn.Linear(28 * 28 * 64, 10)

    def forward(self, x):
      x = self.relu1(self.conv1(x))
      x = self.relu2(self.conv2(x))
      x = x.view(x.size(0), -1)
      x = self.fc(x)
      return x

network = CNN()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)


for epoch in range(1, n_epochs + 1):
    train_loss, train_accuracy = train_epoch(network, trainloader, optimizer)
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}')
train_loss, train_accuracy = test_epoch(network, trainloader)
test_loss, test_accuracy = test_epoch(network, testloader)
print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}')


100%|██████████| 170498071/170498071 [00:03<00:00, 43493492.39it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Example is of length 2
Type of first argument is <class 'torch.Tensor'>
Type of second argument is <class 'torch.Tensor'>
First argument has shape: torch.Size([64, 3, 32, 32])
Second argument has shape torch.Size([64])
Epoch 1, Train Loss: 1.7879, Train Accuracy: 0.37
Epoch 2, Train Loss: 1.5355, Train Accuracy: 0.46
Epoch 3, Train Loss: 1.4273, Train Accuracy: 0.49
Epoch 4, Train Loss: 1.3470, Train Accuracy: 0.52
Epoch 5, Train Loss: 1.2996, Train Accuracy: 0.54
Epoch 6, Train Loss: 1.2534, Train Accuracy: 0.56
Epoch 7, Train Loss: 1.2076, Train Accuracy: 0.58
Epoch 8, Train Loss: 1.1683, Train Accuracy: 0.59
Epoch 9, Train Loss: 1.1372, Train Accuracy: 0.60
Epoch 10, Train Loss: 1.1125, Train Accuracy: 0.61
Train Loss: 0.0016, Train Accuracy: 0.56
Test Loss: 0.1185, Test Accuracy: 0.56
